In [1]:
import pandas as pd
from pandas import DataFrame as df
import country_converter as coco

In [2]:
data = pd.read_csv('time_series_19-covid-Confirmed.csv')

In [3]:
data.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,50,50,53,59,70,75,82,114,147,177
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,502,511,581,639,639,701,773,839,825,878
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,150,150,160,178,178,200,212,226,243,266
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,99,117,129,149,149,197,238,428,566,673


In [4]:
# Select & concat two columns
data = pd.concat([data['Country/Region'], data['3/17/20']], axis=1, keys=['Country/Region', 'Confirmed'])
data.head()

,Country/Region,Confirmed
0,Thailand,177
1,Japan,878
2,Singapore,266
3,Nepal,1
4,Malaysia,673


In [5]:
cc = coco.CountryConverter()

In [6]:
#Standardize country name
standard=cc.convert(names = data['Country/Region'].tolist(), to = 'name_short')

In [7]:
# add a new column that has the standard name
data['Country'] = standard
data.head()

,Country/Region,Confirmed,Country
0,Thailand,177,Thailand
1,Japan,878,Japan
2,Singapore,266,Singapore
3,Nepal,1,Nepal
4,Malaysia,673,Malaysia


In [8]:
# check missing country name - cruise ship is the only one.
data[data['Country'].str.match('not found')]

,Country/Region,Confirmed,Country
165,Cruise Ship,696,not found


In [9]:
# Here we drop the entire row. You can handle it differently if you wish
data = data[~data['Country'].str.contains('not found')]
data[data['Country'].str.match('not found')]

,Country/Region,Confirmed,Country


In [10]:
#Aggregate the case number into a single country since the data is split by regions
aggregations = {
    'Confirmed':'sum'
}
data= data.groupby('Country').agg(aggregations)

In [11]:
coord = pd.read_csv('country_centroids_az8.csv')

In [12]:
coord.head()

,FID,the_geom,FID_,scalerank,featurecla,labelrank,sovereignt,sov_a3,adm0_dif,level,...,region_un,subregion,region_wb,name_len,long_len,abbrev_len,tiny,homepart,Longitude,Latitude
0,country_centroids_az8.1,POINT (-69.9826771125 12.5208803838),0,3,Admin-0 country,5,Netherlands,NL1,1,2,...,Americas,Caribbean,Latin America & Caribbean,5,5,5,4,-99,-69.982677,12.520880
1,country_centroids_az8.2,POINT (66.0047336558 33.8352307278),1,1,Admin-0 country,3,Afghanistan,AFG,0,2,...,Asia,Southern Asia,South Asia,11,11,4,-99,1,66.004734,33.835231
2,country_centroids_az8.3,POINT (17.5373676815 -12.2933605438),2,1,Admin-0 country,3,Angola,AGO,0,2,...,Africa,Middle Africa,Sub-Saharan Africa,6,6,4,-99,1,17.537368,-12.293361
3,country_centroids_az8.4,POINT (-63.0649892654 18.2239595023),3,1,Admin-0 country,6,United Kingdom,GB1,1,2,...,Americas,Caribbean,Latin America & Caribbean,8,8,4,-99,-99,-63.064989,18.223959
4,country_centroids_az8.5,POINT (20.0498339611 41.1424498947),4,1,Admin-0 country,6,Albania,ALB,0,2,...,Europe,Southern Europe,Europe & Central Asia,7,7,4,-99,1,20.049834,41.142450


In [13]:
# again, select useful columns
coord_clean = pd.concat([coord['admin'],coord['Longitude'],coord['Latitude']], axis=1, keys=['Country/Region', 'Longitude','Latitude'])

In [14]:
coord_clean.head()

,Country/Region,Longitude,Latitude
0,Aruba,-69.982677,12.520880
1,Afghanistan,66.004734,33.835231
2,Angola,17.537368,-12.293361
3,Anguilla,-63.064989,18.223959
4,Albania,20.049834,41.142450


In [15]:
standard=cc.convert(names = coord_clean['Country/Region'].tolist(), to = 'name_short')

In [16]:
coord_clean['Country'] = standard

In [17]:
coord_clean[coord_clean['Country'].str.match('not found')]

,Country/Region,Longitude,Latitude,Country
12,Ashmore and Cartier Islands,123.583838,-12.429932,not found
52,CuraÃ§ao,-68.971194,12.195517,not found
99,Indian Ocean Territories,104.851898,-10.647851,not found
111,Siachen Glacier,77.180119,35.392363,not found


In [18]:
# Combine two data set - add longitude & latitude if the country name is a match
# pandas.DataFrame.merge only check for strings that are exactly the same, thus we standard the country name in the first place
combined_data = data.merge(coord_clean,how='left',on='Country')
combined_data.head()

,Country,Confirmed,Country/Region,Longitude,Latitude
0,Afghanistan,22,Afghanistan,66.004734,33.835231
1,Albania,55,Albania,20.049834,41.142450
2,Algeria,60,Algeria,2.617323,28.158938
3,Andorra,39,Andorra,1.560544,42.542291
4,Antigua and Barbuda,1,Antigua and Barbuda,-61.794693,17.277500


In [19]:
# check if there's any NaN (missing) data
combined_data[combined_data.isnull().any(axis=1)]

,Country,Confirmed,Country/Region,Longitude,Latitude
90,Martinique,16,NaN,NaN,NaN


In [20]:
# I just dropped the row - it is the most common approch when handling missing data
# You can add the longitude and latitude manually by googling - if you'd like to keep it 
combined_data.dropna(axis=0, inplace=True) 
combined_data.isnull().any()

Country           False
Confirmed         False
Country/Region    False
Longitude         False
Latitude          False
dtype: bool

In [21]:
combined_data.drop('Country/Region', axis=1)

,Country,Confirmed,Longitude,Latitude
0,Afghanistan,22,66.004734,33.835231
1,Albania,55,20.049834,41.142450
2,Algeria,60,2.617323,28.158938
3,Andorra,39,1.560544,42.542291
4,Antigua and Barbuda,1,-61.794693,17.277500
...,...,...,...,...
147,Uruguay,29,-56.018071,-32.799515
148,Uzbekistan,10,63.140015,41.755542
149,Vatican,1,12.433872,41.901750
150,Venezuela,33,-66.181841,7.124224


In [22]:
combined_data.to_csv('ready_for_import.csv')